<a href="https://colab.research.google.com/github/animeshmodi/HUGGING-FACE-TRANS/blob/main/Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=438038c605ecc1f6e4420ff95bf09a0a64cf041faa0fdcab9a3493b7f3c08166
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


# Installing lib.


In [4]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch
import wikipedia
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the pre-trained BERT model and tokenizer


In [5]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Function to retrieve relevant context from Wikipedia

In [6]:
def retrieve_context(question):
    try:
        # Search for relevant Wikipedia page
        page = wikipedia.page(wikipedia.search(question)[0])
        context = page.content
    except wikipedia.exceptions.PageError:
        context = "No relevant context found on Wikipedia."
    return context

# Function to perform question reformulation

In [7]:

def reformulate_question(question, context):
    # Encode the question and context
    encoding = tokenizer(question, context, return_tensors='pt', padding='max_length', truncation=True)

    # Get the question and context embeddings
    question_embedding = model.bert(**encoding)[0][:, 0, :].detach().numpy()
    context_embedding = model.bert(**encoding)[0][:, 1:, :].mean(dim=1).detach().numpy()

    # Compute the similarity between the question and context embeddings
    similarity_scores = cosine_similarity(question_embedding, context_embedding).squeeze()

    # Reformulate the question based on the most similar context segment
    most_similar_index = np.argmax(similarity_scores)
    reformulated_question = tokenizer.decode(encoding['input_ids'][0][most_similar_index + 1:])

    return reformulated_question

 # Function to answer the question

In [8]:
def answer_question(question, context):
    # Retrieve additional context from Wikipedia if needed
    additional_context = retrieve_context(question)
    context = context + " " + additional_context

    # Reformulate the question based on the context
    reformulated_question = reformulate_question(question, context)

    # Encode the reformulated question and context
    encoding = tokenizer(reformulated_question, context, return_tensors='pt', padding='max_length', truncation=True)

    # Use the BERT model for question answering
    output = model(**encoding)
    answer_start_scores = output.start_logits
    answer_end_scores = output.end_logits

    # Post-process the output to get the answer
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0][answer_start:answer_end]))

    return answer

 Example usage

In [9]:
question = "What is the capital of France?"
context = "France, officially the French Republic, is a transcontinental country spanning Western Europe and overseas regions and territories in the Americas and the Atlantic, Pacific and Indian Oceans."


In [10]:
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Question: What is the capital of France?
Answer: paris
